# Advanced Data Processing for DCA

This notebook demonstrates advanced data processing techniques for preparing production data for decline curve analysis.

## What You'll Learn
- Clean and filter production data
- Handle multi-well datasets
- Calculate derived metrics (water cut, GOR, days online)
- Detect production anomalies
- Prepare data for forecasting
- Use Prophet for seasonal forecasting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from decline_analysis import dca
from decline_analysis.utils import data_processing as dp

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load and Explore Real Bakken Data

In [ ]:
# Load the Bakken well data
df = pd.read_csv('data/bakken_well_production.csv')
df['ReportDate'] = pd.to_datetime(df['ReportDate'])

print("Raw Data Overview:")
print(f"Total records: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")

df.head()

## 2. Data Cleaning and Filtering

In [ ]:
# Remove records with zero or null oil production
df_clean = dp.remove_nan_and_zeroes(df, 'Oil')

print(f"Records before cleaning: {len(df)}")
print(f"Records after cleaning: {len(df_clean)}")
print(f"Records removed: {len(df) - len(df_clean)}")

# Check for any remaining data quality issues
print(f"\nData Quality Checks:")
print(f"  Negative oil values: {(df_clean['Oil'] < 0).sum()}")
print(f"  Negative water values: {(df_clean['Wtr'] < 0).sum()}")
print(f"  Negative gas values: {(df_clean['Gas'] < 0).sum()}")

## 3. Calculate Derived Metrics

In [ ]:
# Calculate days online
df_clean['Online_Date'] = dp.get_grouped_min_max(
    df_clean, 'API_WELLNO', 'ReportDate', 'min'
)
df_clean['Days_Online'] = dp.calculate_days_online(
    df_clean, 'ReportDate', 'Online_Date'
)

# Calculate daily rates
df_clean['oil_rate'] = dp.normalize_production_to_daily(df_clean, 'Oil', 'Days')
df_clean['water_rate'] = dp.normalize_production_to_daily(df_clean, 'Wtr', 'Days')
df_clean['gas_rate'] = dp.normalize_production_to_daily(df_clean, 'Gas', 'Days')

# Calculate water cut and GOR
df_clean['water_cut'] = dp.calculate_water_cut(df_clean, 'Oil', 'Wtr')
df_clean['gor'] = dp.calculate_gor(df_clean, 'Gas', 'Oil')

# Calculate cumulative production
df_clean['cum_oil'] = dp.calculate_cumulative_production(
    df_clean, 'Oil', 'API_WELLNO'
)

print("Derived Metrics Summary:")
print(df_clean[['oil_rate', 'water_cut', 'gor', 'Days_Online', 'cum_oil']].describe().round(2))

df_clean.head()

## 4. Visualize Derived Metrics

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Oil rate vs days online
axes[0, 0].scatter(df_clean['Days_Online'], df_clean['oil_rate'], alpha=0.6)
axes[0, 0].set_xlabel('Days Online')
axes[0, 0].set_ylabel('Oil Rate (bbl/day)')
axes[0, 0].set_title('Oil Rate Decline')
axes[0, 0].grid(True, alpha=0.3)

# Water cut over time
axes[0, 1].plot(df_clean['ReportDate'], df_clean['water_cut'], 'o-', markersize=4)
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Water Cut (%)')
axes[0, 1].set_title('Water Cut Trend')
axes[0, 1].grid(True, alpha=0.3)

# GOR over time
axes[1, 0].plot(df_clean['ReportDate'], df_clean['gor'], 'o-', markersize=4, color='red')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('GOR (mcf/bbl)')
axes[1, 0].set_title('Gas-Oil Ratio')
axes[1, 0].grid(True, alpha=0.3)

# Cumulative oil
axes[1, 1].plot(df_clean['ReportDate'], df_clean['cum_oil'], '-', linewidth=2, color='green')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Cumulative Oil (bbl)')
axes[1, 1].set_title('Cumulative Production')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Detect Production Anomalies

In [ ]:
# Prepare time series
oil_series = df_clean.set_index('ReportDate')['Oil']

# Detect anomalies
anomalies = dp.detect_production_anomalies(oil_series, threshold_std=2.5)

print(f"Anomaly Detection Results:")
print(f"  Total records: {len(oil_series)}")
print(f"  Anomalies detected: {anomalies.sum()}")
print(f"  Anomaly rate: {anomalies.sum() / len(oil_series) * 100:.1f}%")

if anomalies.sum() > 0:
    print(f"\nAnomalous Production Values:")
    print(oil_series[anomalies])

# Visualize anomalies
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(oil_series.index, oil_series.values, 'o-', label='Production', markersize=4)
ax.plot(oil_series[anomalies].index, oil_series[anomalies].values, 'ro', 
        label='Anomalies', markersize=8, zorder=5)
ax.set_xlabel('Date')
ax.set_ylabel('Oil Production (bbl/month)')
ax.set_title('Production with Anomaly Detection')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Prepare Data for DCA (Convenience Function)

In [ ]:
# Get well ID from the data
well_id = df_clean['API_WELLNO'].iloc[0]

# Use convenience function to prepare data
oil_series_clean = dp.prepare_well_data_for_dca(
    df,
    well_id=well_id,
    well_column='API_WELLNO',
    date_column='ReportDate',
    production_column='Oil',
    remove_zeros=True
)

print("Prepared Time Series for DCA:")
print(f"  Well ID: {well_id}")
print(f"  Data points: {len(oil_series_clean)}")
print(f"  Date range: {oil_series_clean.index[0]} to {oil_series_clean.index[-1]}")
print(f"  Total production: {oil_series_clean.sum():,.0f} bbl")
print(f"\nFirst few values:")
print(oil_series_clean.head())

## 7. Get Maximum Initial Production (qi)

In [ ]:
# Calculate qi from first 3 months (handles ramp-up)
qi = dp.get_max_initial_production(
    df_clean,
    n_months=3,
    production_column='Oil',
    date_column='ReportDate'
)

print(f"Initial Production Rate (qi):")
print(f"  Maximum in first 3 months: {qi:,.0f} bbl/month")
print(f"  First month production: {df_clean['Oil'].iloc[0]:,.0f} bbl/month")
print(f"\nUsing qi = {qi:,.0f} bbl/month for decline curve fitting")

# Show first 3 months
print(f"\nFirst 3 months production:")
print(df_clean[['ReportDate', 'Oil']].head(3))

## 8. Run DCA with Prepared Data

In [ ]:
# Generate forecast using prepared data
forecast = dca.forecast(
    oil_series_clean,
    model='arps',
    kind='hyperbolic',
    horizon=24
)

# Evaluate
metrics = dca.evaluate(oil_series_clean, forecast)

print("Forecast Results:")
print(f"  RMSE: {metrics['rmse']:.0f} bbl/month")
print(f"  MAE: {metrics['mae']:.0f} bbl/month")
print(f"  SMAPE: {metrics['smape']:.1f}%")

# Plot
dca.plot(oil_series_clean, forecast, 
         title='Bakken Well Forecast (Processed Data)',
         filename='bakken_forecast_processed.png')

## 9. Prophet Forecasting (Optional)

If you have Prophet installed, you can use it for seasonal forecasting.

In [ ]:
try:
    from decline_analysis import forecast_prophet
    
    # Generate Prophet forecast
    prophet_forecast = forecast_prophet.forecast_prophet(
        oil_series_clean,
        horizon=24,
        yearly_seasonality=False,
        changepoint_prior_scale=0.05
    )
    
    # Evaluate
    prophet_metrics = dca.evaluate(oil_series_clean, prophet_forecast)
    
    print("Prophet Forecast Results:")
    print(f"  RMSE: {prophet_metrics['rmse']:.0f} bbl/month")
    print(f"  MAE: {prophet_metrics['mae']:.0f} bbl/month")
    print(f"  SMAPE: {prophet_metrics['smape']:.1f}%")
    
    # Compare with Arps
    print(f"\nModel Comparison:")
    print(f"  Arps RMSE: {metrics['rmse']:.0f}")
    print(f"  Prophet RMSE: {prophet_metrics['rmse']:.0f}")
    
    if prophet_metrics['rmse'] < metrics['rmse']:
        print(f"  ✓ Prophet performs better by {metrics['rmse'] - prophet_metrics['rmse']:.0f} bbl/month")
    else:
        print(f"  ✓ Arps performs better by {prophet_metrics['rmse'] - metrics['rmse']:.0f} bbl/month")
        
except ImportError:
    print("Prophet not installed. Install with: pip install prophet")
    print("Skipping Prophet forecasting example.")

## Summary

In this notebook, we demonstrated:
1. ✓ Loading and exploring raw production data
2. ✓ Cleaning and filtering data
3. ✓ Calculating derived metrics (water cut, GOR, days online)
4. ✓ Detecting production anomalies
5. ✓ Using convenience functions to prepare data for DCA
6. ✓ Calculating initial production rate (qi)
7. ✓ Running decline curve analysis
8. ✓ Optional: Prophet forecasting for seasonal patterns

## Key Takeaways

- **Data quality matters**: Always clean and validate production data
- **Derived metrics**: Water cut and GOR provide insights into well performance
- **Anomaly detection**: Identify operational issues or data errors
- **Convenience functions**: Use `prepare_well_data_for_dca()` for quick setup
- **Multiple models**: Compare Arps, ARIMA, and Prophet for best results

## Next Steps

- Apply these techniques to your own production data
- Experiment with different anomaly detection thresholds
- Try Prophet with seasonal patterns (gas wells)
- Batch process multiple wells using these utilities